In [12]:
import requests
import random
from typing import Dict, Any, Optional


BASE_URL = "https://berghain.challenges.listenlabs.ai/"  # ← replace with the real host
PLAYER_ID = "5d7389b8-683e-453b-a35f-6d21b1b5e94e"
SCENARIO = 2  # choose 1, 2, or 3

def create_new_game(scenario: int, player_id: str) -> Dict[str, Any]:
    url = f"{BASE_URL}/new-game"
    res = requests.get(url, params={"scenario": scenario, "playerId": player_id}, timeout=30)
    res.raise_for_status()
    data = res.json()
    print("New game:", data["gameId"])
    return data


def decide_and_next(game_id: str, person_index: int, accept: Optional[bool] = None) -> Dict[str, Any]:
    url = f"{BASE_URL}/decide-and-next"
    params = {"gameId": game_id, "personIndex": person_index}
    if accept is not None:
        params["accept"] = str(accept).lower()  # true/false in query string
    res = requests.get(url, params=params, timeout=30)
    res.raise_for_status()
    return res.json()

new_game = create_new_game(SCENARIO, PLAYER_ID)
game_id = new_game["gameId"]
constraints = new_game.get("constraints", [])
stats = new_game.get("attributeStatistics", {})

print(constraints)
print(stats)

New game: 04f514ac-6428-4536-a2e3-d5bc46335262
[{'attribute': 'techno_lover', 'minCount': 650}, {'attribute': 'well_connected', 'minCount': 450}, {'attribute': 'creative', 'minCount': 300}, {'attribute': 'berlin_local', 'minCount': 750}]
{'relativeFrequencies': {'techno_lover': 0.6265000000000001, 'well_connected': 0.4700000000000001, 'creative': 0.06227, 'berlin_local': 0.398}, 'correlations': {'techno_lover': {'techno_lover': 1, 'well_connected': -0.4696169332674324, 'creative': 0.09463317039891586, 'berlin_local': -0.6549403815606182}, 'well_connected': {'techno_lover': -0.4696169332674324, 'well_connected': 1, 'creative': 0.14197259140471485, 'berlin_local': 0.5724067808436452}, 'creative': {'techno_lover': 0.09463317039891586, 'well_connected': 0.14197259140471485, 'creative': 1, 'berlin_local': 0.14446459505650772}, 'berlin_local': {'techno_lover': -0.6549403815606182, 'well_connected': 0.5724067808436452, 'creative': 0.14446459505650772, 'berlin_local': 1}}}


In [13]:
state = decide_and_next(game_id, person_index=0)
if state.get("status") == "failed":
    raise RuntimeError(f"Game failed: {state.get('reason')}")
admitted = state.get("admittedCount", 0)
rejected = state.get("rejectedCount", 0)
next_person = state.get("nextPerson")
total_ct = 0
total_wcb = 0
total_b = 0
total_t = 0
total_c = 0
total_wc = 0
n_accepted = 0
while state.get("status") == "running" and next_person:
        idx = next_person["personIndex"]
        attrs = next_person["attributes"]  # { attributeId: bool }
        t, wc, c, b = attrs.get("techno_lover", False), attrs.get("well_connected", False), attrs.get("creative", False), attrs.get("berlin_local", False)
        if b and t:
            accept = True        
        elif c and t:
            if total_ct < 250 or total_c < 300:
                accept = True
                total_ct += 1
        elif b and wc:
            if total_wcb < 350 and total_wc < 450:
                accept = True
                total_wcb += 1
        elif total_t < 650 and total_b >= 750:
            if t: accept = True
        elif total_t >= 650 and total_b < 750:
            if b: accept = True
        else:
            accept = False
        if accept:
            total_b += b
            total_t += t
            total_c += c
            total_wc += wc
            n_accepted += 1
        state = decide_and_next(game_id, person_index=idx, accept=accept)
        status = state.get("status")
        if status == "failed":
            raise RuntimeError(f"Game failed: {state.get('reason')}")

        admitted = state.get("admittedCount", admitted)
        rejected = state.get("rejectedCount", rejected)
        next_person = state.get("nextPerson")

        print(f"Person {idx}: accept={accept} | admitted={admitted} rejected={rejected}")

        # 5) done
if state.get("status") == "completed":
        print("Game completed.")
        print("Final rejected count:", state.get("rejectedCount"))
else:
        print("Game ended with state:", state)

Person 0: accept=False | admitted=0 rejected=1
Person 1: accept=True | admitted=1 rejected=1
Person 2: accept=False | admitted=1 rejected=2
Person 3: accept=False | admitted=1 rejected=3
Person 4: accept=False | admitted=1 rejected=4
Person 5: accept=True | admitted=2 rejected=4
Person 6: accept=False | admitted=2 rejected=5
Person 7: accept=True | admitted=3 rejected=5
Person 8: accept=True | admitted=4 rejected=5
Person 9: accept=True | admitted=5 rejected=5
Person 10: accept=False | admitted=5 rejected=6
Person 11: accept=True | admitted=6 rejected=6
Person 12: accept=False | admitted=6 rejected=7
Person 13: accept=True | admitted=7 rejected=7
Person 14: accept=True | admitted=8 rejected=7
Person 15: accept=False | admitted=8 rejected=8
Person 16: accept=False | admitted=8 rejected=9
Person 17: accept=True | admitted=9 rejected=9
Person 18: accept=False | admitted=9 rejected=10
Person 19: accept=False | admitted=9 rejected=11
Person 20: accept=False | admitted=9 rejected=12
Person 2

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))